#LBP CK+ dataset

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

def calculate_lbp_pixel(center, pixels):
    binary_values = [1 if pixel >= center else 0 for pixel in pixels]
    lbp_value = sum([binary_values[i] * (2 ** i) for i in range(8)])
    return lbp_value

def compute_lbp(image):
    lbp_image = np.zeros_like(image, dtype=np.uint8)
    rows, cols = image.shape

    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            center = image[i, j]
            pixels = [image[i - 1, j - 1], image[i - 1, j], image[i - 1, j + 1],
                      image[i, j + 1], image[i + 1, j + 1], image[i + 1, j],
                      image[i + 1, j - 1], image[i, j - 1]]
            lbp_value = calculate_lbp_pixel(center, pixels)
            lbp_image[i, j] = lbp_value

    return lbp_image.ravel()

def read_images_and_compute_lbp(base_path):
    lbp_features = []
    labels = []
    expressions = os.listdir(base_path)

    for expression in expressions:
        expression_path = os.path.join(base_path, expression)
        if os.path.isdir(expression_path):
            images = [f for f in os.listdir(expression_path) if f.endswith('.png')]
            for image_file in images:
                image_path = os.path.join(expression_path, image_file)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                lbp_features.append(compute_lbp(image))
                labels.append(expression)

    return np.array(lbp_features), np.array(labels)

# Paths to train, val, and test folders
train_path = '/content/drive/MyDrive/EE511_Assignments/D22180_Nandani_Sharma_Assignment2/CK+/train'
val_path = '/content/drive/MyDrive/EE511_Assignments/D22180_Nandani_Sharma_Assignment2/CK+/val'
test_path = '/content/drive/MyDrive/EE511_Assignments/D22180_Nandani_Sharma_Assignment2/CK+/test'

# Read images and compute LBP features for train, val, and test sets
train_data, train_labels = read_images_and_compute_lbp(train_path)
val_data, val_labels = read_images_and_compute_lbp(val_path)
test_data, test_labels = read_images_and_compute_lbp(test_path)

# Create DataFrames
train_df = pd.DataFrame(train_data)
train_df['label'] = train_labels

val_df = pd.DataFrame(val_data)
val_df['label'] = val_labels

test_df = pd.DataFrame(test_data)
test_df['label'] = test_labels

# Save as CSV files
train_df.to_csv('/content/drive/MyDrive/EE511_Assignments/D22180_Nandani_Sharma_Assignment2/CK+/Train.csv', index=False)
val_df.to_csv('/content/drive/MyDrive/EE511_Assignments/D22180_Nandani_Sharma_Assignment2/CK+/Val.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/EE511_Assignments/D22180_Nandani_Sharma_Assignment2/CK+/Test.csv', index=False)

print("CSV files generated successfully.")


CSV files generated successfully.


In [ ]:
class CustomSVM:
    def __init__(self, learning_rate=0.01, num_epochs=1000, C=1.0, kernel='linear'):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.C = C  # Regularization parameter
        self.kernel = kernel
        self.weights = None
        self.bias = None

    def linear_kernel(self, X):
        return np.dot(X, X.T)

    def rbf_kernel(self, X, gamma=0.1):
        n_samples, n_features = X.shape
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i, j] = np.exp(-gamma * np.linalg.norm(X[i] - X[j])**2)
        return K


    def train(self, X, y):
        n_samples, n_features = X.shape
        if self.kernel == 'linear':
            K = self.linear_kernel(X)
        elif self.kernel == 'rbf':
            K = self.rbf_kernel(X)
        else:
            raise ValueError("Invalid kernel type.")

        # Convert labels to numerical values: 'non-face' -> -1, 'face' -> 1
        numerical_labels = np.where(y == 'non-face', -1, 1)

        # Initialize weights and bias
        self.weights = np.zeros(n_features)  # <-- Initialize with the shape of features
        self.bias = 0

        for epoch in range(self.num_epochs):
            for i in range(n_samples):
                # Calculate margin
                margin = numerical_labels[i] * (np.sum(self.weights * K[i]) + self.bias)

                # Update weights and bias if margin is less than 1
                if margin < 1:
                    self.weights = self.weights + self.learning_rate * (self.C * numerical_labels[i] * K[i] - 2 * self.C * self.weights)
                    self.bias = self.bias + self.learning_rate * (self.C * numerical_labels[i])

    def predict(self, X):
        n_samples, _ = X.shape
        predictions = []
        if self.kernel == 'linear':
            K = np.dot(X, X.T)
        elif self.kernel == 'rbf':
            K = self.rbf_kernel(X)
        else:
            raise ValueError("Invalid kernel type.")

        for i in range(n_samples):
            prediction = np.sum(self.weights * K[i]) + self.bias
            predictions.append(np.sign(prediction))
        return np.array(predictions)

In [ ]:
# Assuming you have train_df, val_df, and test_df DataFrames
# train_df, val_df, and test_df contain your LBP features with the 'label' column indicating the class

# Extract features and labels for training, validation, and test sets
X_train = train_df.drop('label', axis=1).values
y_train = train_df['label'].values

X_val = val_df.drop('label', axis=1).values
y_val = val_df['label'].values

X_test = test_df.drop('label', axis=1).values
y_test = test_df['label'].values

# Standardize features
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)
X_test_std = scaler.transform(X_test)

# Combine training and validation data for training
X_train_combined_std = np.vstack((X_train_std, X_val_std))
y_train_combined = np.hstack((y_train, y_val))

# Instantiate and train the custom SVM classifier
svm_classifier = CustomSVM(learning_rate=0.01, num_epochs=1000, C=1.0, kernel='linear')
svm_classifier.train(X_train_combined_std, y_train_combined)

# Predict and evaluate on test data
y_pred_custom = svm_classifier.predict(X_test_std)

# Evaluate accuracy without using built-in functions
accuracy_custom_svm = np.mean(y_pred_custom == y_test)
print(f'Accuracy of Custom SVM classifier on Test.csv: {accuracy_custom_svm:.2f}')


In [ ]:
import os
import shutil
import random

# Path to the CK+ dataset
dataset_path = '/content/drive/MyDrive/FER_GCN/CK+48/train'

# Path to store the splitted dataset
output_path = '/content/drive/MyDrive/E511_Assignments/D22180_Nandani_Sharma_Assignment2/CK+'

# List the expression folders in the dataset
expression_folders = ['anger', 'contempt', 'fear', 'happy', 'sadness', 'surprise']

# Create train and validation directories in the output folder for each expression
for expression_folder in expression_folders:
    os.makedirs(os.path.join(output_path, 'train', expression_folder), exist_ok=True)
    os.makedirs(os.path.join(output_path, 'val', expression_folder), exist_ok=True)

    expression_images_path = os.path.join(dataset_path, expression_folder)
    images = [f for f in os.listdir(expression_images_path) if f.endswith('.png')]
    random.shuffle(images)  # Shuffle the images randomly

    # Define the ratio for splitting (e.g., 80% train, 20% validation)
    split_ratio = 0.8

    # Split the images based on the split_ratio
    split_index = int(len(images) * split_ratio)
    train_images = images[:split_index]
    val_images = images[split_index:]

    # Move images to train folder
    for image in train_images:
        src = os.path.join(expression_images_path, image)
        dst = os.path.join(output_path, 'train', expression_folder, image)
        shutil.copy(src, dst)

    # Move images to validation folder
    for image in val_images:
        src = os.path.join(expression_images_path, image)
        dst = os.path.join(output_path, 'val', expression_folder, image)
        shutil.copy(src, dst)

print("Dataset split into train and val folders successfully.")


Dataset split into train and val folders successfully.


In [ ]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Load validation data
val_df = pd.read_csv('/content/drive/MyDrive/EE511_Assignments/D22180_Nandani_Sharma_Assignment2/CK+/Val.csv')
X_val = val_df.drop('label', axis=1)
y_val = val_df['label']

# Standardize features
scaler = StandardScaler()
X_val_std = scaler.fit_transform(X_val)

# Define the parameter grid for GridSearchCV
param_grid = {
    'estimator__C': [1, 10, 100],  # You can specify other values as well
    'estimator__kernel': ['linear', 'rbf']  # Choose appropriate kernels
}

# Perform GridSearchCV to find the best parameters
svm = SVC(probability=True)
ovr_svm = OneVsRestClassifier(svm)
grid_search = GridSearchCV(ovr_svm, param_grid, cv=5)
grid_search.fit(X_val_std, y_val)

# Get the best parameters
best_C = grid_search.best_params_['estimator__C']
best_kernel = grid_search.best_params_['estimator__kernel']

# Train One-vs-Rest SVM classifier with best parameters using the combined train and validation data
X_train_combined = np.concatenate((train_df.drop('label', axis=1), X_val))
y_train_combined = np.concatenate((train_df['label'], y_val))
X_train_combined_std = scaler.transform(X_train_combined)

svm_classifier = OneVsRestClassifier(SVC(C=best_C, kernel=best_kernel, probability=True))
svm_classifier.fit(X_train_combined_std, y_train_combined)

# Load test data
test_df = pd.read_csv('/content/drive/MyDrive/EE511_Assignments/D22180_Nandani_Sharma_Assignment2/CK+/Test.csv')
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

# Standardize test data
X_test_std = scaler.transform(X_test)

# Predict and evaluate on test data
y_pred = svm_classifier.predict(X_test_std)
accuracy_svm = accuracy_score(y_test, y_pred)
print(f'Accuracy of One-vs-Rest SVM classifier on Test.csv: {accuracy_svm:.2f}')


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Accuracy of One-vs-Rest SVM classifier on Test.csv: 1.00


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
# Load validation data
val_df = pd.read_csv('/content/drive/MyDrive/EE511_Assignments/D22180_Nandani_Sharma_Assignment2/CK+/Val.csv')
X_val = val_df.drop('label', axis=1)
y_val = val_df['label']

# Standardize features
scaler = StandardScaler()
X_val_std = scaler.fit_transform(X_val)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [3, 5, 7],  # You can specify other values as well
    'metric': ['euclidean', 'manhattan']  # Choose appropriate distance metrics
}

# Perform GridSearchCV to find the best parameters
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_val_std, y_val)

# Get the best parameters
best_k = grid_search.best_params_['n_neighbors']
best_metric = grid_search.best_params_['metric']

# Train KNN classifier with best parameters using the combined train and validation data
X_train_combined = np.concatenate((train_df.drop('label', axis=1), X_val))
y_train_combined = np.concatenate((train_df['label'], y_val))
X_train_combined_std = scaler.transform(X_train_combined)

knn_classifier = KNeighborsClassifier(n_neighbors=best_k, metric=best_metric)
knn_classifier.fit(X_train_combined_std, y_train_combined)

# Load test data
test_df = pd.read_csv('/content/drive/MyDrive/EE511_Assignments/D22180_Nandani_Sharma_Assignment2/CK+/Test.csv')
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

# Standardize test data
X_test_std = scaler.transform(X_test)

# Predict and evaluate on test data
y_pred = knn_classifier.predict(X_test_std)
accuracy_knn = accuracy_score(y_test, y_pred)
print(f'Accuracy of KNN classifier on Test.csv: {accuracy_knn:.2f}')


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Accuracy of KNN classifier on Test.csv: 0.80
